<a href="https://colab.research.google.com/github/deanhansen/colab/blob/main/ISLR_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as nphttps://drive.google.com/drive/search?q=owner%3Ame%20(type%3Aapplication%2Fvnd.google.colaboratory%20%7C%7C%20type%3Aapplication%2Fvnd.google.colab)&authuser=0
import pandas as pd
from matplotlib.pyplot import subplots
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor as VIF
from statsmodels.stats.anova import anova_lm
from ISLP import load_data
from ISLP.models import (ModelSpec as MS, summarize, poly)

In [ ]:
Boston = load_data("Boston")
Boston.keys

<bound method NDFrame.keys of         crim    zn  indus  chas    nox     rm   age     dis  rad  tax  \
0    0.00632  18.0   2.31     0  0.538  6.575  65.2  4.0900    1  296   
1    0.02731   0.0   7.07     0  0.469  6.421  78.9  4.9671    2  242   
2    0.02729   0.0   7.07     0  0.469  7.185  61.1  4.9671    2  242   
3    0.03237   0.0   2.18     0  0.458  6.998  45.8  6.0622    3  222   
4    0.06905   0.0   2.18     0  0.458  7.147  54.2  6.0622    3  222   
..       ...   ...    ...   ...    ...    ...   ...     ...  ...  ...   
501  0.06263   0.0  11.93     0  0.573  6.593  69.1  2.4786    1  273   
502  0.04527   0.0  11.93     0  0.573  6.120  76.7  2.2875    1  273   
503  0.06076   0.0  11.93     0  0.573  6.976  91.0  2.1675    1  273   
504  0.10959   0.0  11.93     0  0.573  6.794  89.3  2.3889    1  273   
505  0.04741   0.0  11.93     0  0.573  6.030  80.8  2.5050    1  273   

     ptratio  lstat  medv  
0       15.3   4.98  24.0  
1       17.8   9.14  21.6  
2       1

In [ ]:
X = pd.DataFrame({'intercept': np.ones(Boston.shape[0]),
'lstat': Boston['lstat'], 'ptratio': Boston['ptratio']})
y = Boston['medv']
model = sm.OLS(y, X)
results = model.fit()

In [ ]:
design = MS(['lstat'])
design = design.fit(Boston)
X = design.transform(Boston)
X[:4]

,intercept,lstat
0,1.0,4.98
1,1.0,9.14
2,1.0,4.03
3,1.0,2.94


In [ ]:
new_df = pd.DataFrame({'lstat':[5, 10, 15]})
newX = design.transform(new_df)
newX

,intercept,lstat
0,1.0,5
1,1.0,10
2,1.0,15


In [ ]:
new_predictions = results.get_prediction(newX);
new_predictions.predicted_mean

array([29.80359411, 25.05334734, 20.30310057])

In [ ]:
summarize(results)

,coef,std err,t,P>|t|
intercept,54.0468,2.242,24.105,0.0
lstat,-0.8202,0.039,-21.120,0.0
ptratio,-1.1453,0.128,-8.941,0.0


In [ ]:
class std_errors:
  def get_usual_standard_errors(X, results):
    X_transpose = np.transpose(X)
    M = np.dot(X_transpose, X)
    M_inv = np.linalg.inv(M)
    return np.sqrt(np.diag(results.mse_resid * M_inv))

  def get_newey_west_standard_errors(X, results):
    """
    X : a model matrix with column of 1's for intercept term
    results : fitted linear regression object from statsmodel API
    """
    X_transpose = np.transpose(X)
    M = np.dot(X_transpose, X)
    M_inv = np.linalg.inv(M)
    res = results.resid
    res_squared = np.multiply(res,res)

    T = np.zeros((3,3))

    for row in range(X.shape[0]):
      T += np.dot(res_squared[row], np.outer(X.iloc[row],np.transpose(X.iloc[row])))
    return np.sqrt(np.diag(M_inv.dot(T).dot(M_inv)))

In [ ]:
np.dot(np.dot(X, np.transpose(X)), X)

array([[ 175269.251   , 2390619.890034, 3285444.55582 ],
       [ 225249.693   , 3137456.069062, 4225925.96226 ],
       [ 192533.1735  , 2591897.352149, 3607202.51727 ],
       ...,
       [ 232724.318   , 3151244.623812, 4361227.35676 ],
       [ 238102.376   , 3240925.508784, 4462935.32032 ],
       [ 247065.806   , 3390393.650404, 4632448.59292 ]])

In [ ]:
std_errors.get_newey_west_standard_errors(X,results)

array([2.68838822, 0.04681258, 0.14821194])

In [ ]:
std_errors.get_usual_standard_errors(X,results)

array([2.24216585, 0.03883463, 0.12809583])

In [ ]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   medv   R-squared:                       0.607
Model:                            OLS   Adj. R-squared:                  0.605
Method:                 Least Squares   F-statistic:                     387.9
Date:                Sat, 09 Sep 2023   Prob (F-statistic):          1.22e-102
Time:                        20:46:20   Log-Likelihood:                -1604.2
No. Observations:                 506   AIC:                             3214.
Df Residuals:                     503   BIC:                             3227.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
intercept     54.0468      2.242     24.105      0.000      49.642      58.452
lstat         -0.8202      0.039    -21.120      0.000      -0.896      -0.744
ptratio       -1.1453      0.128     -8.941      0.000      -1.397      -0.894
==============================================================================
Omnibus:                      150.452   Durbin-Watson:                   0.994
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              403.510
Skew:                           1.462   Prob(JB):                     2.39e-88
Kurtosis:                       6.253   Cond. No.                         201.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""